In [ ]:
# | default_exp nets/perceiver_3d

# Imports

In [ ]:
# | export

import torch
from einops import rearrange, repeat
from huggingface_hub import PyTorchModelHubMixin
from torch import nn

from vision_architectures.layers.attention import Attention1DWithMLP, Attention1DWithMLPConfig
from vision_architectures.layers.embeddings import AbsolutePositionEmbeddings3D
from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.custom_base_model import CustomBaseModel, model_validator

# Config

In [ ]:
# | export


class Perceiver3DChannelMappingConfig(CustomBaseModel):
    in_channels: int | set[int]
    out_channels: int


class Perceiver3DEncoderEncodeConfig(Attention1DWithMLPConfig):
    dim: int
    num_latent_tokens: int
    num_layers: int


class Perceiver3DEncoderProcessConfig(Attention1DWithMLPConfig):
    dim: int
    num_layers: int


class Perceiver3DEncoderConfig(CustomBaseModel):
    encode: Perceiver3DEncoderEncodeConfig
    process: Perceiver3DEncoderProcessConfig

    @property
    def dim(self):
        return self.encode.dim

    @model_validator(mode="after")
    def validate(self):
        super().validate()
        assert self.encode.dim == self.process.dim, "encode and process dims must be equal"
        return self


class Perceiver3DDecoderConfig(Attention1DWithMLPConfig):
    dim: int
    num_layers: int
    out_channels: int


class Perceiver3DConfig(Perceiver3DEncoderConfig):
    decode: Perceiver3DDecoderConfig

    @model_validator(mode="after")
    def validate(self):
        super().validate()
        assert self.encode.dim == self.decode.dim, "encode and decode dims must be equal"
        return self

In [ ]:
test_config = Perceiver3DConfig.model_validate(
    {
        "encode": {
            "dim": 384,
            "num_latent_tokens": 1024,
            "num_layers": 1,
            "num_heads": 16,
        },
        "process": {
            "dim": 384,
            "num_layers": 3,
            "num_heads": 16,
        },
        "decode": {
            "dim": 384,
            "out_channels": 1,
            "num_layers": 3,
            "num_heads": 16,
        },
    }
)
test_config.dim

384

# Utility functions

In [ ]:
# | export


def unfold_with_rollover_1d(x: torch.Tensor, window_size: int | None, stride: int | None):
    # x: (b, T, dim)
    if window_size is None or stride is None:
        return x.unsqueeze(0)
    total_len = x.shape[1]
    num_windows = (total_len + stride - 1) // stride  # Number of windows needed
    pad_len = max(0, window_size + (num_windows - 1) * stride - total_len)  # How many elements are missing
    if pad_len > 0:
        x = torch.cat([x, x[:, :pad_len]], dim=1)  # Rollover padding
    x = x.unfold(1, window_size, stride)
    x = rearrange(x, "b num_windows dim window_size -> num_windows b window_size dim")
    # (num_windows, b, window_size, dim)
    return x

In [ ]:
x = torch.arange(4).reshape(1, 4, 1)
unfolded = unfold_with_rollover_1d(x, 3, 2)
unfolded.shape, unfolded


(
    torch.Size([2, 1, 3, 1]),
    tensor([[[[0],
          [1],
          [2]]],


        [[[2],
          [3],
          [0]]]])
)

In [ ]:
# | export


def unfold_with_rollover_3d_with_mask(
    x: torch.Tensor, window_size: tuple[int, int, int] | None, stride: tuple[int, int, int] | None
):
    if window_size is None or stride is None:
        return x.unsqueeze(0)

    b, c, d, h, w = x.shape
    w_d, w_h, w_w = window_size
    s_d, s_h, s_w = stride

    # Calculate number of windows and required padding
    n_d = (d + s_d - 1) // s_d
    n_h = (h + s_h - 1) // s_h
    n_w = (w + s_w - 1) // s_w

    # Calculate padding with rollover
    pad_d = max(0, w_d + (n_d - 1) * s_d - d)
    pad_h = max(0, w_h + (n_h - 1) * s_h - h)
    pad_w = max(0, w_w + (n_w - 1) * s_w - w)

    # Apply rollover padding efficiently
    if pad_d > 0:
        x = torch.cat([x, x[:, :, :pad_d]], dim=2)
    if pad_h > 0:
        x = torch.cat([x, x[:, :, :, :pad_h]], dim=3)
    if pad_w > 0:
        x = torch.cat([x, x[:, :, :, :, :pad_w]], dim=4)

    # Extract patches using stride tricks
    # This approach avoids creating intermediate tensors
    patches = x.unfold(2, w_d, s_d).unfold(3, w_h, s_h).unfold(4, w_w, s_w)

    # Rearrange dimensions using einops for clarity and efficiency
    patches = rearrange(
        patches, "b c d_windows h_windows w_windows w_d w_h w_w -> (d_windows h_windows w_windows) b c w_d w_h w_w"
    )

    # Padded shape
    padded_d, padded_h, padded_w = x.shape[2], x.shape[3], x.shape[4]

    # Generate positional information for each fold
    # Create meshgrid for all window positions
    d_indices = torch.arange(n_d, device=x.device)
    h_indices = torch.arange(n_h, device=x.device)
    w_indices = torch.arange(n_w, device=x.device)

    # Calculate starting indices for each window
    d_starts = d_indices * s_d
    h_starts = h_indices * s_h
    w_starts = w_indices * s_w

    # Generate all possible window starting positions
    d_pos, h_pos, w_pos = torch.meshgrid(d_starts, h_starts, w_starts, indexing="ij")
    positions = torch.stack([d_pos, h_pos, w_pos], dim=-1).reshape(-1, 3)

    # Generate usage count mask
    usage_mask = torch.zeros(padded_d, padded_h, padded_w, device=x.device)

    # For each position, increment the count for all pixels in that window
    for pos in positions:
        pos_d, pos_h, pos_w = pos
        # Handle potential out-of-bounds for the last window
        end_d = min(pos_d + w_d, padded_d)
        end_h = min(pos_h + w_h, padded_h)
        end_w = min(pos_w + w_w, padded_w)

        usage_mask[pos_d:end_d, pos_h:end_h, pos_w:end_w] += 1

    # Crop usage_mask back to original dimensions (without padding)
    usage_mask = usage_mask[:d, :h, :w]

    # Repeat usage mask for batch and channel dimensions
    usage_mask = usage_mask.expand(b, c, d, h, w)

    return patches, positions, usage_mask


def fold_back_3d(
    patches: torch.Tensor,
    positions: torch.Tensor,
    usage_mask: torch.Tensor,
    output_shape: tuple,
    window_size: tuple[int, int, int],
    aggregation_mode: str = "mean",
):
    b, c, d, h, w = output_shape
    w_d, w_h, w_w = window_size

    # Initialize output tensor
    output = torch.zeros(output_shape, dtype=patches.dtype, device=patches.device)

    # Handle case where we want sum instead of mean
    if aggregation_mode == "sum":
        usage_mask = torch.ones_like(usage_mask)

    # Fold each patch back to its position
    for i, pos in enumerate(positions):
        pos_d, pos_h, pos_w = pos
        # Get the patch
        patch = patches[i]

        # Handle window dimensions (might be smaller at edges)
        valid_w_d = min(w_d, d - pos_d) if pos_d < d else 0
        valid_w_h = min(w_h, h - pos_h) if pos_h < h else 0
        valid_w_w = min(w_w, w - pos_w) if pos_w < w else 0

        if valid_w_d <= 0 or valid_w_h <= 0 or valid_w_w <= 0:
            continue

        # Only add the valid portion of the patch
        output[:, :, pos_d : pos_d + valid_w_d, pos_h : pos_h + valid_w_h, pos_w : pos_w + valid_w_w] += patch[
            :, :, :valid_w_d, :valid_w_h, :valid_w_w
        ]

    # Average according to usage count
    if aggregation_mode == "mean":
        # Avoid division by zero
        mask = (usage_mask > 0).float()
        output = output / (usage_mask + (1 - mask))

    return output

# Architecture

### Channel mapping

In [ ]:
# | export


class Perceiver3DChannelMapping(nn.Module):
    def __init__(self, config: Perceiver3DChannelMappingConfig = {}, **kwargs):
        super().__init__()

        self.config = Perceiver3DChannelMappingConfig.model_validate(config | kwargs)

        self.in_channels = self.config.in_channels
        self.out_channels = self.config.out_channels

        if isinstance(self.in_channels, int):
            self.in_channels = {self.in_channels}

        self.mappers = nn.ModuleDict()
        for in_channels in self.in_channels:
            self.mappers[str(in_channels)] = nn.Conv3d(in_channels, self.out_channels, kernel_size=1)

    def forward(self, x: torch.Tensor):
        # x: (b, in_channels, z, y, x)

        in_channels = x.shape[1]
        if in_channels not in self.in_channels:
            raise ValueError(f"Input channels {in_channels} not in {self.in_channels}")

        mapper = self.mappers[str(in_channels)]
        x = mapper(x)

        return x

In [ ]:
test = Perceiver3DChannelMapping(in_channels={12, 24, 48}, out_channels=24)
for in_channels in [12, 24, 48]:
    x = torch.randn(1, in_channels, 4, 4, 4)
    print(test(x).shape)

torch.Size([1, 24, 4, 4, 4])
torch.Size([1, 24, 4, 4, 4])
torch.Size([1, 24, 4, 4, 4])


### Encoder

In [ ]:
# | export


class Perceiver3DEncoderEncode(nn.Module):
    def __init__(
        self,
        config: Perceiver3DEncoderEncodeConfig | Perceiver3DEncoderConfig = {},
        channel_mapping: Perceiver3DChannelMapping | None = None,
        checkpointing_level: int = 0,
        **kwargs
    ):
        super().__init__()

        if isinstance(config, Perceiver3DEncoderConfig):
            config = config.encode
        if "encode" in config:
            config = config["encode"]

        self.config = Perceiver3DEncoderEncodeConfig.model_validate(config | kwargs)

        dim = self.config.dim
        num_latent_tokens = self.config.num_latent_tokens
        num_layers = self.config.num_layers

        self.latent_tokens = nn.Parameter(torch.empty(num_latent_tokens, dim), requires_grad=True)
        nn.init.xavier_uniform_(self.latent_tokens)

        self.channel_mapping = channel_mapping

        self.cross_attention = nn.ModuleList(
            [
                Attention1DWithMLP(self.config.model_dump(), checkpointing_level=checkpointing_level)
                for _ in range(num_layers)
            ]
        )

        self.checkpointing_level1 = ActivationCheckpointing(1, checkpointing_level)
        self.checkpointing_level4 = ActivationCheckpointing(4, checkpointing_level)

    def _forward(
        self,
        x: torch.Tensor | list[torch.Tensor],
        sliding_window: int | None = None,
        sliding_stride: int | None = None,
        return_all: bool = False,
    ) -> torch.Tensor | dict[str, torch.Tensor]:
        # x: [(b, in_channels, z, y, x), ...]

        # Prepare keys and values
        def prepare_keys_values(x: torch.Tensor | list[torch.Tensor]):
            if not isinstance(x, list):
                x = [x]
            # x is now a list of tensors
            if self.channel_mapping is None:
                kv = x
            else:
                kv = []
                for i in range(len(x)):
                    mapped = self.channel_mapping(x[i])  # modifying in-place leads to errors when checkpointing
                    mapped = rearrange(mapped, "b d z y x -> b (z y x) d")
                    kv.append(mapped)
            kv = torch.cat(kv, dim=1)
            return kv

        kv = self.checkpointing_level1(prepare_keys_values, x)
        # (b, num_kv_tokens, dim)

        # Prepare queries
        b = kv.shape[0]
        q = repeat(self.latent_tokens, "t d -> b t d", b=b)
        # (b, num_latent_tokens, dim)

        # Prepare sliding window
        kv_windows = unfold_with_rollover_1d(kv, sliding_window, sliding_stride)
        # (num_windows, b, window_size, dim)

        # Perform attention
        embeddings = []
        for cross_attention_layer in self.cross_attention:
            embedding = torch.zeros_like(q)
            for kv_window in kv_windows:
                embedding_window = cross_attention_layer(q, kv_window, kv_window)
                embedding = embedding + embedding_window
            q = embedding  # To pass to the next layer
            embeddings.append(embedding)
        # (b, num_latent_tokens, dim)

        return_value = embeddings[-1]
        if return_all:
            return_value = {
                "embeddings": return_value,
                "all_embeddings": embeddings,
            }
        return return_value

    def forward(
        self,
        x: torch.Tensor | list[torch.Tensor],
        sliding_window: int | None = None,  # Sliding window may be beneficial during inference time
        sliding_stride: int | None = None,
        return_all: bool = False,
    ):
        return self.checkpointing_level4(self._forward, x, sliding_window, sliding_stride, return_all)

In [ ]:
%%time

test = Perceiver3DEncoderEncode(test_config, Perceiver3DChannelMapping(in_channels=512, out_channels=384))
display(test)
o = test(torch.randn(2, 512, 4, 4, 4))
display(o.shape)


Perceiver3DEncoderEncode(
  (channel_mapping): Perceiver3DChannelMapping(
    (mappers): ModuleDict(
      (512): Conv3d(512, 384, kernel_size=(1, 1, 1), stride=(1, 1, 1))
    )
  )
  (cross_attention): ModuleList(
    (0): Attention1DWithMLP(
      (attn): Attention1D(
        (W_q): Linear(in_features=384, out_features=384, bias=True)
        (W_k): Linear(in_features=384, out_features=384, bias=True)
        (W_v): Linear(in_features=384, out_features=384, bias=True)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention1DMLP(
        (dense1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=1536, out_features=384, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((384,), eps=1e-06, ele

torch.Size([2, 1024, 384])

CPU times: user 1.21 s, sys: 859 ms, total: 2.07 s
Wall time: 71 ms


In [ ]:
%%time

test.eval()
with torch.no_grad():
    o = test(torch.randn(2, 512, 4, 4, 4), sliding_window=8, sliding_stride=8)
display(o.shape)

torch.Size([2, 1024, 384])

CPU times: user 3.63 s, sys: 1.01 s, total: 4.64 s
Wall time: 76.1 ms


In [ ]:
# | export


class Perceiver3DEncoderProcess(nn.Module):
    def __init__(
        self,
        config: Perceiver3DEncoderProcessConfig | Perceiver3DEncoderConfig = {},
        checkpointing_level: int = 0,
        **kwargs
    ):
        super().__init__()

        if isinstance(config, Perceiver3DEncoderConfig):
            config = config.process
        if "process" in config:
            config = config["process"]

        self.config = Perceiver3DEncoderProcessConfig.model_validate(config | kwargs)

        num_layers = self.config.num_layers

        self.self_attention = nn.ModuleList(
            [
                Attention1DWithMLP(self.config.model_dump(), checkpointing_level=checkpointing_level)
                for _ in range(num_layers)
            ]
        )

        self.checkpointing_level4 = ActivationCheckpointing(4, checkpointing_level)

    def _forward(self, qkv, return_all: bool = False) -> torch.Tensor | dict[str, torch.Tensor]:
        # qkv: (b, num_tokens, dim)

        embeddings = []
        for self_attention_layer in self.self_attention:
            embedding = self_attention_layer(qkv, qkv, qkv)
            qkv = embedding  # To pass to the next layer
            embeddings.append(embedding)
        # (b, num_tokens, dim)

        return_value = embeddings[-1]
        if return_all:
            return_value = {
                "embeddings": return_value,
                "all_embeddings": embeddings,
            }

        return return_value

    def forward(self, q: torch.Tensor, return_all: bool = False):
        return self.checkpointing_level4(self._forward, q, return_all)

In [ ]:
test = Perceiver3DEncoderProcess(dim=1024, num_heads=16, num_layers=2)
display(test)
o = test(torch.randn(2, 512, 1024))
display(o.shape)


Perceiver3DEncoderProcess(
  (self_attention): ModuleList(
    (0-1): 2 x Attention1DWithMLP(
      (attn): Attention1D(
        (W_q): Linear(in_features=1024, out_features=1024, bias=True)
        (W_k): Linear(in_features=1024, out_features=1024, bias=True)
        (W_v): Linear(in_features=1024, out_features=1024, bias=True)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention1DMLP(
        (dense1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=4096, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
    )
  )
)

torch.Size([2, 512, 1024])

In [ ]:
# | export


class Perceiver3DEncoder(nn.Module, PyTorchModelHubMixin):
    def __init__(
        self,
        config: Perceiver3DEncoderConfig = {},
        channel_mapping: Perceiver3DChannelMapping | None = None,
        checkpointing_level: int = 0,
        **kwargs,
    ):
        super().__init__()

        self.config = Perceiver3DEncoderConfig.model_validate(config | kwargs)

        self.encode = Perceiver3DEncoderEncode(config.encode, channel_mapping, checkpointing_level)
        self.process = Perceiver3DEncoderProcess(config.process, checkpointing_level)

        self.checkpointing_level5 = ActivationCheckpointing(5, checkpointing_level)

    def _forward(
        self,
        x,
        sliding_window: int | None = None,
        sliding_stride: int | None = None,
        return_all: bool = False,
    ) -> torch.Tensor | dict[str, torch.Tensor]:
        # x: (b, in_channels, z, y, x)

        return_value = {}

        encode_embeddings = self.encode(x, sliding_window, sliding_stride, return_all=True)["all_embeddings"]
        return_value["encode_embeddings"] = encode_embeddings
        embeddings = encode_embeddings[-1]
        # (b, num_tokens, dim)

        process_embeddings = self.process(embeddings, return_all=True)["all_embeddings"]
        return_value["process_embeddings"] = process_embeddings
        embeddings = process_embeddings[-1]

        return_value["embeddings"] = embeddings

        if not return_all:
            return_value = embeddings

        return return_value

    def forward(
        self,
        x: torch.Tensor,
        sliding_window: int | None = None,
        sliding_stride: int | None = None,
        return_all: bool = False,
    ):
        return self.checkpointing_level5(self._forward, x, sliding_window, sliding_stride, return_all)

In [ ]:
test = Perceiver3DEncoder(test_config, Perceiver3DChannelMapping(in_channels={512, 256}, out_channels=384))
display(test)
o = test(torch.randn(2, 256, 4, 4, 4))
display(o.shape)


Perceiver3DEncoder(
  (encode): Perceiver3DEncoderEncode(
    (channel_mapping): Perceiver3DChannelMapping(
      (mappers): ModuleDict(
        (512): Conv3d(512, 384, kernel_size=(1, 1, 1), stride=(1, 1, 1))
        (256): Conv3d(256, 384, kernel_size=(1, 1, 1), stride=(1, 1, 1))
      )
    )
    (cross_attention): ModuleList(
      (0): Attention1DWithMLP(
        (attn): Attention1D(
          (W_q): Linear(in_features=384, out_features=384, bias=True)
          (W_k): Linear(in_features=384, out_features=384, bias=True)
          (W_v): Linear(in_features=384, out_features=384, bias=True)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (layernorm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Attention1DMLP(
          (dense1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (dense2): Linear(in_features=1

torch.Size([2, 1024, 384])

### Decoder

In [ ]:
# | export


class Perceiver3DDecoder(nn.Module, PyTorchModelHubMixin):
    def __init__(
        self,
        config: Perceiver3DDecoderConfig | Perceiver3DConfig = {},
        position_embeddings: AbsolutePositionEmbeddings3D = None,
        checkpointing_level: int = 0,
        **kwargs,
    ):
        super().__init__()

        if isinstance(config, Perceiver3DConfig):
            config = config.decode
        if "decode" in config:
            config = config["decode"]

        self.config = Perceiver3DDecoderConfig.model_validate(config | kwargs)

        dim = self.config.dim
        num_layers = self.config.num_layers

        self.empty_token = nn.Parameter(torch.randn(dim, 1) * 0.02, requires_grad=True)
        # Initialized with gaussian for robust training stability

        self.position_embeddings = position_embeddings

        self.cross_attention = nn.ModuleList(
            [
                Attention1DWithMLP(config.model_dump(), checkpointing_level=checkpointing_level)
                for _ in range(num_layers)
            ]
        )

        self.channel_mapping = Perceiver3DChannelMapping(in_channels=dim, out_channels=self.config.out_channels)

        self.checkpointing_level4 = ActivationCheckpointing(4, checkpointing_level)

    #     # Prepare sliding windows
    #     q_windows, positions, usage_mask = unfold_with_rollover_3d_with_mask(q, sliding_window, sliding_stride)
    #     # (num_windows, b, dim, window_size_z, window_size_y, window_size_x)

    #     # Perform attention
    #     outputs = []
    #     for cross_attention_layer in self.cross_attention:
    #         output = torch.zeros_like(q)
    #         for q_window in q_windows:
    #             output_window = cross_attention_layer(q_window, kv, kv)
    #             output = output + output_window
    #     q = rearrange(q, "b d z y x -> b (z y x) d")
    #     # (b, num_output_tokens, dim)
    #     outputs = [q]
    #     for cross_attention_layer in self.cross_attention:
    #         q = outputs[-1]
    #         outputs.append(cross_attention_layer(q, kv, kv))
    #     # (b, num_output_tokens, dim)

    #     output = outputs[-1]
    #     output = rearrange(
    #         output,
    #         "b (z y x) d -> b d z y x",
    #         z=out_shape[0],
    #         y=out_shape[1],
    #         x=out_shape[2],
    #     )
    #     # (b, dim, z, y, x)

    #     output = self.channel_mapping(output)
    #     # (b, out_channels, z, y, x)

    #     return_value = output
    #     if return_all:
    #         return_value = {
    #             "output": output,
    #             "all_outputs": outputs,
    #         }

    #     return return_value

    def _forward(
        self,
        kv,
        out_shape: tuple[int, int, int],
        sliding_window: tuple[int, int, int] | None = None,
        sliding_stride: tuple[int, int, int] | None = None,
        return_all: bool = False,
    ) -> torch.Tensor | dict[str, torch.Tensor]:
        # kv: (b, num_tokens, dim)

        b = kv.shape[0]

        q = repeat(
            self.empty_token,
            "d 1 -> b d z y x",
            b=b,
            z=out_shape[0],
            y=out_shape[1],
            x=out_shape[2],
        )
        # (b, dim, z, y, x)

        if self.position_embeddings is not None:
            q = q + self.position_embeddings(batch_size=b, grid_size=out_shape, device=q.device)

        # Conditional execution based on sliding window parameters
        if sliding_window is not None and sliding_stride is not None:
            # Apply sliding window processing
            q_windows, positions, usage_mask = unfold_with_rollover_3d_with_mask(q, sliding_window, sliding_stride)
            # q_windows: (num_windows, b, dim, window_size_z, window_size_y, window_size_x)

            # Process each window with attention
            num_windows = q_windows.shape[0]
            window_shape = q_windows.shape[-3:]  # (window_size_z, window_size_y, window_size_x)

            # Initialize output tensor for each layer's output
            outputs = []
            current_windows = q_windows

            # Apply cross-attention to each window sequentially through all layers
            for layer_idx, cross_attention_layer in enumerate(self.cross_attention):
                processed_windows = torch.zeros_like(current_windows)

                for window_idx in range(num_windows):
                    # Extract current window
                    window = current_windows[window_idx]  # (b, dim, w_z, w_y, w_x)

                    # Reshape for attention operation
                    flat_window = rearrange(window, "b d z y x -> b (z y x) d")

                    # Apply cross-attention
                    attended_window = cross_attention_layer(flat_window, kv, kv)

                    # Reshape back to 3D
                    processed_window = rearrange(
                        attended_window,
                        "b (z y x) d -> b d z y x",
                        z=window_shape[0],
                        y=window_shape[1],
                        x=window_shape[2],
                    )

                    # Store processed window
                    processed_windows[window_idx] = processed_window

                # Update current windows for next layer
                current_windows = processed_windows

                # Fold windows back to full volume for this layer's output
                folded_output = fold_back_3d(
                    processed_windows, positions, usage_mask, (b, q.shape[1], out_shape[0], out_shape[1], out_shape[2])
                )

                # Store layer output for return_all
                flat_output = rearrange(
                    folded_output, "b d z y x -> b (z y x) d", z=out_shape[0], y=out_shape[1], x=out_shape[2]
                )
                outputs.append(flat_output)

            # Final output is from the last layer
            output = folded_output

        else:
            # Original processing without sliding windows
            q = rearrange(q, "b d z y x -> b (z y x) d")
            # (b, num_output_tokens, dim)
            outputs = [q]
            for cross_attention_layer in self.cross_attention:
                q = outputs[-1]
                outputs.append(cross_attention_layer(q, kv, kv))
            # (b, num_output_tokens, dim)

            output = outputs[-1]
            output = rearrange(
                output,
                "b (z y x) d -> b d z y x",
                z=out_shape[0],
                y=out_shape[1],
                x=out_shape[2],
            )
            # (b, dim, z, y, x)

        # Apply channel mapping to get final output
        output = self.channel_mapping(output)
        # (b, out_channels, z, y, x)

        return_value = output
        if return_all:
            return_value = {
                "output": output,
                "all_outputs": outputs,
            }

        return return_value

    def forward(
        self,
        kv: torch.Tensor,
        out_shape: tuple[int, int, int],
        return_all: bool = False,
    ):
        return self.checkpointing_level4(self._forward, kv, out_shape, return_all)

In [ ]:
test = Perceiver3DDecoder(test_config, AbsolutePositionEmbeddings3D(dim=test_config.dim))
display(test)
o = test(torch.randn(2, 1024, 384), (20, 20, 20))
display(o.shape)


Perceiver3DDecoder(
  (position_embeddings): AbsolutePositionEmbeddings3D()
  (cross_attention): ModuleList(
    (0-2): 3 x Attention1DWithMLP(
      (attn): Attention1D(
        (W_q): Linear(in_features=384, out_features=384, bias=True)
        (W_k): Linear(in_features=384, out_features=384, bias=True)
        (W_v): Linear(in_features=384, out_features=384, bias=True)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (layernorm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Attention1DMLP(
        (dense1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (dense2): Linear(in_features=1536, out_features=384, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (layernorm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
    )
  )
  (channel_mapping): Perceiver3DChannelMapping(
    (mappers): Modul

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 test = Perceiver3DDecoder(test_config, AbsolutePositionEmbeddings3D(dim=test_config.dim)     │
│   2 display(test)                                                                                │
│ ❱ 3 o = test(torch.randn(2, 1024, 384), (20, 20, 20))                                            │
│   4 display(o.shape)                                                                             │
│   5                                                                                              │
│                                                                                                  │
│ /home/ubuntu/miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:1736 in          │
│ _wrapped_call_impl                                                                               │
│                                                                                                  │
│   1733 │   │   if self._compiled_call_impl is not None:                                          │
│   1734 │   │   │   return self._compiled_call_impl(*args, **kwargs)  # type: ignore[misc]        │
│   1735 │   │   else:                                                                             │
│ ❱ 1736 │   │   │   return self._call_impl(*args, **kwargs)                                       │
│   1737 │                                                                                         │
│   1738 │   # torchrec tests the code consistency with the following code                         │
│   1739 │   # fmt: off                                                                            │
│                                                                                                  │
│ /home/ubuntu/miniconda3/lib/python3.10/site-packages/torch/nn/modules/module.py:1747 in          │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1744 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1745 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1746 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1747 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1748 │   │                                                                                     │
│   1749 │   │   result = None                                                                     │
│   1750 │   │   called_always_called_hooks = set()                                                │
│                                                                                                  │
│ in forward:189                                                                                   │
│                                                                                                  │
│   186 │   │   out_shape: tuple[int, int, int],                                                   │
│   187 │   │   return_all: bool = False,                                                          │
│   188 │   ):                                                                                     │
│ ❱ 189 │   │   return self.checkpointing_level4(self._forward, kv, out_shape, return_all)         │
│   190                                                                                            │
│                                                                                                  │
│ /home/ubuntu/projects/vision_architectures/vision_architect

# nbdev

In [ ]:
!nbdev_export